In [1]:
import pytz
from datetime import datetime
import calendar

import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px

In [2]:
def monthly_energy(
    year: int, power_W: float, freq_minutes: int, label: str
) -> pd.DataFrame:

    tz_str = "Europe/Vienna"
    tz = pytz.timezone(tz_str)

    starttime = datetime(
        year=year,
        month=1,
        day=1,
    )
    endtime = datetime(
        year=year + 1,
        month=1,
        day=1,
    )

    times = pd.date_range(
        f"{starttime:%Y-%m-%d %H:%M}",
        f"{endtime:%Y-%m-%d %H:%M}",
        freq=f"{freq_minutes}min",
        tz=tz,
    )[:-1]

    pwr = np.ones_like(times) * power_W  # from pvsim...

    df_times = pd.DataFrame(dict(pwr=pwr, month=[t.month for t in times]), index=times)

    pwr_mean_W = df_times.groupby("month").mean().pwr.values
    pwr_count_h = df_times.groupby("month").count().pwr.values * freq_minutes / 60
    e_kWh = pwr_mean_W * pwr_count_h / 1000

    months = [calendar.month_abbr[m + 1] for m in range(12)]

    df_result = pd.DataFrame(data={f"{label}": e_kWh}, index=months)
    return df_result

In [3]:
p1_energy = monthly_energy(year=2022, power_W=100, freq_minutes=60, label="panel1")
p2_energy = monthly_energy(year=2022, power_W=130, freq_minutes=60, label="panel2")
p3_energy = monthly_energy(year=2022, power_W=140, freq_minutes=60, label="panel3")

result = pd.concat([p1_energy, p2_energy, p3_energy], axis=1)

In [4]:
fig = px.bar(result,barmode='group',text_auto='.2f')
fig.update_traces(textfont_size=20, textangle=0, textposition="outside")
fig.show()

In [5]:
# https://plotly.com/python/bar-charts/

fig = go.Figure()
for col in result.columns:
    fig.add_trace(
        go.Bar(
            name=f'{col} ({result[col].sum():.1f} kWh/Y)',
            x=result.index,
            y=result[col],
            hovertemplate="%{x}: %{y:.1f} kWh",#%{_xother}",
            #hovertemplate="%{y:.1f} kWh%{_xother}",
        )
    )

fig.add_trace(
    go.Bar(
        name=f'<b>total ({result.sum().sum():.1f} kWh/Y)</b>',
        x=result.index,
        y=result.sum(axis=1),
        hovertemplate="%{x}: %{y:.1f} kWh",#%{_xother}",
        #hovertemplate="%{y:.1f} kWh%{_xother}",
        marker_color='black'
    )
)
# fig.add_annotation(
#     text='Annual Sums:<br>'+ '<br>'.join([f'+{result[col].sum():.2f}kWh ({col})' for col in result]) +'<br>= '+f'{result.sum().sum():.2f} kWh (total)',
#     showarrow=False,
#     xref='paper',
#     yref='paper',
#     x=0,
#     y=1,
#     bordercolor='black',
#     bgcolor='white',
#     borderwidth=1,
#     align='left',
#     )

fig.update_layout(
    #barmode='stack',
    barmode='group',
    margin=dict(l=10, r=10, t=50, b=5),
    title="Monthly Energy Yield",
    yaxis_title='Energy [kWh]',
    legend=dict(
        x=0.0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0.9)',
        bordercolor='rgba(255, 255, 255, 1)',
    )
    )
fig.show()